In [24]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.optimize import minimize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [26]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [27]:
#Scale for better intepretation
data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [28]:
#Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

In [29]:
data = data.sort_values(['Year', 'ID']).reset_index(drop=True)

# Outside share

In [30]:
data.loc[data['Year'] == 2013, 'Market_share'] = data.loc[data['Year'] == 2013, 'Sales'] / 180632
data.loc[data['Year'] == 2014, 'Market_share'] = data.loc[data['Year'] == 2014, 'Sales'] / 188406
data.loc[data['Year'] == 2015, 'Market_share'] = data.loc[data['Year'] == 2015, 'Sales'] / 206653
data.loc[data['Year'] == 2016, 'Market_share'] = data.loc[data['Year'] == 2016, 'Sales'] / 222471
data.loc[data['Year'] == 2017, 'Market_share'] = data.loc[data['Year'] == 2017, 'Sales'] / 221471
data.loc[data['Year'] == 2018, 'Market_share'] = data.loc[data['Year'] == 2018, 'Sales'] / 252328
data.loc[data['Year'] == 2019, 'Market_share'] = data.loc[data['Year'] == 2019, 'Sales'] / 258727
data.loc[data['Year'] == 2020, 'Market_share'] = data.loc[data['Year'] == 2020, 'Sales'] / 230060
data.loc[data['Year'] == 2021, 'Market_share'] = data.loc[data['Year'] == 2021, 'Sales'] / 222210
data.loc[data['Year'] == 2022, 'Market_share'] = data.loc[data['Year'] == 2022, 'Sales'] / 181030
data.loc[data['Year'] == 2023, 'Market_share'] = data.loc[data['Year'] == 2023, 'Sales'] / 203690

In [31]:
data['outside_share'] = 1 - data.groupby('Year')['Market_share'].transform('sum')
data[['Market_share', 'outside_share']].describe()

,Market_share,outside_share
count,334.000000,334.000000
mean,0.002128,0.840911
std,0.005618,0.108763
min,0.000004,0.696180
25%,0.000123,0.696180
50%,0.000715,0.830851
75%,0.002161,0.938460
max,0.088139,0.997525


# Pure Logit

In [32]:
OLS = sm.ols('np.log(Market_share/outside_share) ~ Price + Range + HP + Chargetime + China', data).fit(cov_type='HC3')
OLS.summary()#.tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     np.log(Market_share / outside_share)   R-squared:                       0.282
Model:                                              OLS   Adj. R-squared:                  0.271
Method:                                   Least Squares   F-statistic:                     35.32
Date:                                  Tue, 04 Jun 2024   Prob (F-statistic):           7.05e-29
Time:                                          18:30:30   Log-Likelihood:                -657.51
No. Observations:                                   334   AIC:                             1327.
Df Residuals:                                       328   BIC:                             1350.
Df Model:                                             5                                         
Covariance Type:                                    HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -8.5912      0.592    -14.515      0.000      -9.751      -7.431
Price         -0.0242      0.005     -4.892      0.000      -0.034      -0.014
Range          0.1124      0.013      8.861      0.000       0.088       0.137
HP            -0.0280      0.011     -2.460      0.014      -0.050      -0.006
Chargetime    -0.0402      0.009     -4.442      0.000      -0.058      -0.022
China         -0.9174      0.307     -2.992      0.003      -1.518      -0.316
==============================================================================
Omnibus:                       38.751   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.788
Skew:                          -0.882   Prob(JB):                     2.55e-11
Kurtosis:                       3.630   Cond. No.                         450.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Willingness to pay

In [33]:
alpha = OLS.params[1] # Price coefficient
beta = list(OLS.params[:1]) + list(OLS.params[2:]) # Other coefficients
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(OLS.params.index[2:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[2:][i], beta_alpha_ratio[i])

W2P: Range 4.648382589502988
W2P: HP -1.1584169816671883
W2P: Chargetime -1.6643290194043667
W2P: China -37.940704167097024


NOTE: English delimiter\
25,926 DKK for every 10 increase in Range\
515 DKK for every 10 increase in HP\
-4,093 DKK for every 10 increase in Chargetime\
-691,420 DKK for Chinese cars OR -69,000 DKK for Chinese cars???

# Substitution patterns

In [34]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [35]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,1.236915,1.258914,2.219064,0.346092,0.42206,0.70475,0.243917,0.232432,3.148824,...,0.304777,0.181784,0.156147,0.074749,0.557406,2.037225,0.392208,0.209423,0.44308,2.971753
U6,0.808463,1.0,1.017785,1.794031,0.279802,0.34122,0.569765,0.197198,0.187912,2.545707,...,0.246401,0.146966,0.126239,0.060432,0.450642,1.647021,0.317086,0.169311,0.358214,2.402552
e-tron,0.794335,0.982526,1.0,1.762681,0.274913,0.335257,0.559808,0.193752,0.184629,2.501222,...,0.242096,0.144398,0.124033,0.059376,0.442768,1.61824,0.311545,0.166352,0.351954,2.360569
e-tron GT,0.45064,0.557404,0.567317,1.0,0.155963,0.190197,0.317589,0.109919,0.104743,1.418987,...,0.137345,0.081919,0.070366,0.033685,0.25119,0.918056,0.176745,0.094374,0.19967,1.339192
Q4 e-tron,2.889406,3.573951,3.637514,6.411779,1.0,1.219502,2.03631,0.704776,0.67159,9.098232,...,0.880626,0.525248,0.451172,0.215981,1.610573,5.886371,1.133249,0.605108,1.280239,8.586602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.490864,0.607157,0.617955,1.089258,0.169884,0.207174,0.345936,0.11973,0.114092,1.545644,...,0.149604,0.089231,0.076647,0.036692,0.273611,1.0,0.192521,0.102798,0.217492,1.458726
C40,2.549666,3.153721,3.20981,5.657873,0.882419,1.076112,1.796878,0.621908,0.592623,8.02845,...,0.777081,0.463489,0.398123,0.190586,1.4212,5.194243,1.0,0.533959,1.129707,7.576978
EX30,4.775025,5.906301,6.011346,10.596088,1.652597,2.015346,3.365201,1.164711,1.109867,15.035712,...,1.45532,0.868024,0.745606,0.356929,2.661629,9.727799,1.872804,1.0,2.11572,14.190194


In [36]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, OLS)
marginal_effects

,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-1.571991,-0.004424,0.020566,-0.005125,-0.007364,-0.167866
Leaf_2013,-0.996402,-0.002804,0.013036,-0.003249,-0.004667,-0.106402
Zoe_2013,-1.104799,-0.00311,0.014454,-0.003602,-0.005175,-0.117977
Fortwo_2013,-0.08042,-0.000226,0.001052,-0.000262,-0.000377,-0.008588
Model S_2013,-2.049067,-0.005767,0.026808,-0.006681,-0.009599,-0.218811
...,...,...,...,...,...,...
up!_2023,-0.017631,-0.00005,0.000231,-0.000057,-0.000083,-0.001883
C40_2023,-0.090789,-0.000256,0.001188,-0.000296,-0.000425,-0.009695
EX30_2023,-0.168427,-0.000474,0.002204,-0.000549,-0.000789,-0.017986


In [37]:
marginal_effects[OLS.params.index].mean()

Intercept    -0.244070
Price        -0.000687
Range         0.003193
HP           -0.000796
Chargetime   -0.001143
China        -0.026063
dtype: float64

In [38]:
elasticity = fun.elasticity(logit_data, OLS)
elasticity

,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-6.519731,-0.458752,2.516296,-0.354993,-0.54973,-0.0
Leaf_2013,-7.440711,-0.531283,3.192996,-0.356619,-1.498754,-0.0
Zoe_2013,-7.289004,-0.355221,3.480736,-0.318454,-1.912069,-0.0
Fortwo_2013,-8.509986,-0.504082,1.413978,-0.221969,-2.391814,-0.0
Model S_2013,-5.216549,-1.056478,4.156336,-1.148048,-0.733081,-0.0
...,...,...,...,...,...,...
up!_2023,-8.573506,-0.421929,2.871498,-0.226421,-1.927734,-0.0
C40_2023,-8.499405,-1.030235,5.181846,-1.114006,-1.114792,-0.0
EX30_2023,-8.419308,-0.872613,5.232149,-0.735672,-1.104287,-0.0


In [39]:
elasticity[OLS.params.index].mean()

Intercept    -8.308231
Price        -1.100176
Range         4.280556
HP           -0.681927
Chargetime   -1.331029
China        -0.106754
dtype: float64

# Cost

In [40]:
fun.cost_firm(logit_data, alpha)
logit_data = fun.markup(logit_data)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,outside_share,CCP,Model_year,firm_cost,markup%
0,10,2013,0.000006,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.997525,0.241113,I3_2013,15.028518,66.350398
1,132,2013,0.001168,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.997525,0.133912,Leaf_2013,19.830919,27.926496
2,158,2013,0.000509,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.997525,0.151571,Zoe_2013,11.046634,56.744577
3,167,2013,0.000006,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.997525,0.009450,Fortwo_2013,20.654930,1.892138
4,173,2013,0.000620,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.997525,0.392801,Model S_2013,55.711588,29.158606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.000388,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.696180,0.002056,up!_2023,12.076742,44.784909
330,188,2023,0.002538,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.696180,0.010682,C40_2023,41.406375,4.009322
331,189,2023,0.000020,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.696180,0.020005,EX30_2023,35.164385,4.721012
332,190,2023,0.009647,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.696180,0.009455,XC40_2023,42.266545,3.927728


In [41]:
cost_side = sm.ols('np.log(firm_cost) ~ Range + HP + Chargetime', logit_data).fit(cov_type='HC3')
cost_side.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(firm_cost)   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     148.4
Date:                Tue, 04 Jun 2024   Prob (F-statistic):           6.88e-61
Time:                        18:30:31   Log-Likelihood:                -119.84
No. Observations:                 334   AIC:                             247.7
Df Residuals:                     330   BIC:                             262.9
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.0708      0.113     27.064      0.000       2.848       3.293
Range          0.0072      0.003      2.631      0.009       0.002       0.013
HP             0.0214      0.002     12.817      0.000       0.018       0.025
Chargetime    -0.0081      0.002     -4.060      0.000      -0.012      -0.004
==============================================================================
Omnibus:                       17.993   Durbin-Watson:                   1.446
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.135
Skew:                           0.485   Prob(JB):                     2.57e-05
Kurtosis:                       3.760   Cond. No.                         333.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Analysis on subsample

### The subsample consists of the 2023 market with:
The 5 highest market share models: Model Y, Model 3, Enyag iV, ID.4 and Q4 e-tron\
The 5 highest market share chinese models: 4, Euniq6, Atto 3, Marvel R and Dolphin\
The highest ccp model: Ocean\
The highest ccp chinese model: Seal\
The highest ccp korean model (so we have models outside EU, US and china): Ioniq 6\
Polestar 2 as a random interest


In [42]:
analysis_data = logit_data[logit_data['Year']==2023].copy()
analysis_data = analysis_data[analysis_data['Model'].isin(['Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
analysis_data.reset_index(drop=True, inplace=True)
analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,outside_share,CCP,Model_year,firm_cost,markup%
0,5,2023,0.011532,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,C,DE,2349,1.0,0,0.69618,0.012105,Q4 e-tron_2023,64.728471,1.740866
1,17,2023,0.001090,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,C,CN,222,1.0,1,0.69618,0.004165,Atto 3_2023,30.196267,2.661697
2,18,2023,0.000997,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,C,CN,203,1.0,1,0.69618,0.006506,Dolphin_2023,22.519783,3.569010
3,20,2023,0.000157,BYD,Seal,54.0,32.062000,52.2,38,Sedan,D,CN,32,1.0,1,0.69618,0.006113,Seal_2023,31.258267,2.571266
4,49,2023,0.001424,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,D,DK,290,1.0,0,0.69618,0.047030,Ocean_2023,58.292944,3.336554
5,69,2023,0.000972,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,D,KR,198,1.0,0,0.69618,0.066872,Ioniq 6_2023,38.789489,12.476775
6,100,2023,0.001213,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,M,CN,247,1.0,1,0.69618,0.001311,Euniq6_2023,37.539246,1.095998
7,117,2023,0.003643,MG,4,42.7,26.589191,24.1,41,Hatchback,C,CN,742,1.0,1,0.69618,0.003815,4_2023,26.100150,1.873709
8,119,2023,0.001021,MG,Marvel R,38.8,30.235650,17.7,38,SUV,C,CN,208,1.0,1,0.69618,0.003042,Marvel R_2023,29.746609,1.644022
9,149,2023,0.009927,Polestar,2,51.3,38.989190,41.5,28,Liftback,D,SE,2022,1.0,0,0.69618,0.019278,2_2023,38.191907,2.087570


In [43]:
cross_elasticity = fun.cross_elasticity_1(analysis_data, OLS)
cross_elasticity[:50]

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:195: PerformanceWarning: indexing past lexsort depth may impact performance.
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]


Cross_Elasticity
Model_year     Model_year                                
Q4 e-tron_2023 Q4 e-tron_2023 Intercept          0.103997
                              Price              0.019276
                              Range             -0.067486
                              HP                 0.009528
                              Chargetime          0.01364
                              China                   0.0
               Atto 3_2023    Intercept          0.035781
                              Price              0.003122
                              Range             -0.019661
                              HP                 0.002345
                              Chargetime         0.006202
                              China              0.003821
               Dolphin_2023   Intercept          0.055896
                              Price              0.003669
                              Range             -0.031226
                              HP                 0.001695
                              Chargetime         0.010473
                              China              0.005969
               Seal_2023      Intercept          0.052517
                              Price              0.004739
                              Range             -0.037102
                              HP                 0.008938
                              Chargetime         0.009348
                              China              0.005608
               Ocean_2023     Intercept          0.404043
                              Price              0.068502
                              Range             -0.370029
                              HP                 0.074167
                              Chargetime         0.066243
                              China                   0.0
               Ioniq 6_2023   Intercept          0.574508
                              Price              0.070547
                              Range             -0.426176
                              HP                  0.05994
                              Chargetime         0.043059
                              China                   0.0
               Euniq6_2023    Intercept          0.011259
                              Price              0.001203
                              Range             -0.005215
                              HP                 0.000639
                              Chargetime         0.002373
                              China              0.001202
               4_2023         Intercept          0.032777
                              Price              0.002453
                              Range             -0.018311
                              HP                 0.002575
                              Chargetime         0.006295
                              China                0.0035
               Marvel R_2023  Intercept          0.026133
                              Price              0.002224

# Nash Equilibrium on subsample

In [44]:
NE_data = logit_data[logit_data['Year']==2023].copy()
NE_data.reset_index(drop=True, inplace=True)
X_ne = NE_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]

NE_analysis_data = NE_data[NE_data['Model'].isin(['Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
NE_analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,outside_share,CCP,Model_year,firm_cost,markup%
4,5,2023,0.011532,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,C,DE,2349,1.0,0,0.69618,0.012105,Q4 e-tron_2023,64.728471,1.740866
13,17,2023,0.001090,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,C,CN,222,1.0,1,0.69618,0.004165,Atto 3_2023,30.196267,2.661697
14,18,2023,0.000997,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,C,CN,203,1.0,1,0.69618,0.006506,Dolphin_2023,22.519783,3.569010
16,20,2023,0.000157,BYD,Seal,54.0,32.062000,52.2,38,Sedan,D,CN,32,1.0,1,0.69618,0.006113,Seal_2023,31.258267,2.571266
26,49,2023,0.001424,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,D,DK,290,1.0,0,0.69618,0.047030,Ocean_2023,58.292944,3.336554
32,69,2023,0.000972,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,D,KR,198,1.0,0,0.69618,0.066872,Ioniq 6_2023,38.789489,12.476775
43,100,2023,0.001213,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,M,CN,247,1.0,1,0.69618,0.001311,Euniq6_2023,37.539246,1.095998
54,117,2023,0.003643,MG,4,42.7,26.589191,24.1,41,Hatchback,C,CN,742,1.0,1,0.69618,0.003815,4_2023,26.100150,1.873709
56,119,2023,0.001021,MG,Marvel R,38.8,30.235650,17.7,38,SUV,C,CN,208,1.0,1,0.69618,0.003042,Marvel R_2023,29.746609,1.644022
74,149,2023,0.009927,Polestar,2,51.3,38.989190,41.5,28,Liftback,D,SE,2022,1.0,0,0.69618,0.019278,2_2023,38.191907,2.087570


In [45]:
car1 = 4
car2 = 13

p1_data = NE_data.loc[car1, 'Price'] 
p2_data = NE_data.loc[car2, 'Price'] 

c1 = NE_data.loc[car1, 'firm_cost']
c2 = NE_data.loc[car2, 'firm_cost']

def market_shares(p1, p2): 
    x2 = NE_data.copy() # copy to avoid writing to the original data 
    fun.set_car_price(x2, p1, car1) # Set new price for car 1
    fun.set_car_price(x2, p2, car2) # set new price for car 2
    ccp = fun.ccp(alpha, beta, x2, X_ne) # compute the CCPs for the new prices
    s1 = ccp[car1] 
    s2 = ccp[car2] 
    return s1, s2

def profit(p, s, c): 
    return s * (p - c)

def profit1(p1, p2): 
    s1, s2 = market_shares(p1, p2)
    pi1 = profit(p1, s1, c1)
    return pi1

def profit2(p2, p1): 
    s1, s2 = market_shares(p1, p2)
    pi2 = profit(p2, s2, c2)
    return pi2

In [46]:
p2 = 0 # initial guess

for i in range(10):
    f = lambda p: -profit1(p1=p, p2=p2)
    res = minimize(f, x0=p2)
    p1 = res['x'][0]
    print(f'BR1(p2={p2:6.4f}) = {p1:6.4f}')

    f = lambda p: -profit2(p1=p1, p2=p)
    res = minimize(f, x0=p2)
    p2 = res['x'][0]
    print(f'BR2(p1={p1:6.4f}) = {p2:6.4f}')

BR1(p2=0.0000) = 106.2140
BR2(p1=106.2140) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
BR1(p2=71.5264) = 106.2730
BR2(p1=106.2730) = 71.5264
